In [1]:
from __future__ import division
from nltk.stem import WordNetLemmatizer

import numpy as np
import pandas
import seaborn as sea

import keras
import scipy
from sklearn import svm

import matplotlib.pyplot as plt

import nltk
from nltk import word_tokenize

from sklearn.preprocessing import StandardScaler

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

/home/philippe/Code/where-is-brian/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package punkt to /home/philippe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/philippe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/philippe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
english = pandas.read_csv("train_cap2018.csv")

In [3]:
target = english['level1']
target.head()
print(target.shape)

# data
data = english.loc[:, "sentences":"Yule's K"]
print(data.shape)

# words_data : 
text = english.loc[:, "fulltext"]
words_data = text.map(lambda t: word_tokenize(t))


# print(data.loc[3]["fulltext"][0])
# sentence = data[:3]["fulltext"][0]
# sentence = data.as_matrix()[27309, 0]
# print(sentence)
# words = word_tokenize(sentence)


(27310,)
(27310, 6)


In [4]:
tagged = nltk.pos_tag(words_data[27304])

print(words_data[27305])

print(len(set(words_data[27305])))

print(len(words_data[27305]))
print(len(set(words_data[27305])) / len(words_data[27305]))
print(len(set(words_data[3000])) / len(words_data[3000]))
print(len(set(words_data[2])) / len(words_data[2]))


def lexical_diversity(text):
    return len(set(text)) / len(text)


ld = words_data.map(lambda t: lexical_diversity(t))



['Hi', 'my', 'name', "'s", 'Jean-Marc', '.', 'I', "'m", 'a', 'management', 'controller', '.', 'I', 'work', 'in', 'an', 'office', '.', 'My', 'favorite', 'day', 'in', 'Sunday', '.', 'I', 'get', 'up', 'at', 'eight', "o'clock", 'in', 'the', 'morning', 'on', 'Sunday', ',', 'have', 'a', 'shower', ',', 'brush', 'my', 'teeth', 'and', 'go', 'running', '.', 'I', 'have', 'lunch', 'at', 'twelve', "o'clock", 'in', 'the', 'afternoon', '.', 'I', 'go', 'swimming', 'in', 'the', 'evening', ',', 'I', 'meet', 'friends', '.', 'We', 'eat', 'in', 'a', 'restaurant', 'then', 'go', 'to', 'the', 'movies', 'and', 'watch', 'a', 'movie', '.', 'I', 'go', 'to', 'bed', 'at', 'eleven', "o'clock", '.']
52
91
0.5714285714285714
0.6238532110091743
0.6938775510204082


In [5]:
#

#words_lem = map(lambda s: set(map(wordnet_lemmatizer.lemmatize, s)), words_data)
#words_target = list(zip(target, words_lem))
#df = pandas.DataFrame.from_records(words_target, columns=['level', 'words'])

#print(df)
#df['words'].groupby(df['level'])

pdf = pandas.DataFrame({'level': target, 'words': words_data})
grouped = pdf.groupby(['level'], sort=False).sum()
grouped

,words
level,
C2,"[The, Eiffel, Tower, is, a, symbol, of, domina..."
C1,"[Since, our, meeting, I, 've, heard, your, req..."
B2,"[At, the, beginning, of, the, metting, the, CE..."
B1,"[Hello, my, friend, ,, I, would, like, to, tel..."
A2,"[Dear, friends, ,, excuse, me, but, I, could, ..."
A1,"[Hello, My, name, 's, David, ., I, 'm, from, F..."


In [6]:
lemmatizer = WordNetLemmatizer()
grouped_in_set = grouped.applymap(lambda l: set(map(lemmatizer.lemmatize, l)))
grouped_in_set

,words
level,
C2,"{month, had, look, style, useful, meant, 150, ..."
C1,"{saved, month, flirting, non-negligible, eithe..."
B2,"{to.The, encovered, neer, month, family.The, s..."
B1,"{disappears, country.Next, month, saved, eagle..."
A2,"{either, stopped, elisabeth, Novembre, Sheen, ..."
A1,"{Febuary, Lisle, neer, month, foorty, saved, D..."


In [7]:
diff_words_count = grouped_in_set.applymap(lambda s: len({w for w in s if w[0].islower()}))
diff_words = grouped_in_set.applymap(lambda s: {w for w in s if w[0].islower()})

In [8]:
words_lem = words_data.apply(lambda l: set(map(lemmatizer.lemmatize, l)))

In [13]:
diff_words_count["words"]["C2"]

1759

In [9]:
print(len(words_lem[1] & diff_words["words"]["C2"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["C1"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["B2"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["B1"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["A2"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["A1"])/len(words_lem[1]))

print(len(words_lem[27306] & diff_words["words"]["C2"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["C1"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["B2"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["B1"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["A2"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["A1"])/len(words_lem[27306]))

dist_C2 = pandas.DataFrame({'C2': words_lem.map(lambda t: len(t & diff_words["words"]["C2"])/len(t)),
                            'C1': words_lem.map(lambda t: len(t & diff_words["words"]["C1"])/len(t)),
                            'B2': words_lem.map(lambda t: len(t & diff_words["words"]["B2"])/len(t)),
                            'B1': words_lem.map(lambda t: len(t & diff_words["words"]["B1"])/len(t)),
                            'A2': words_lem.map(lambda t: len(t & diff_words["words"]["A2"])/len(t)),
                            'A1': words_lem.map(lambda t: len(t & diff_words["words"]["A1"])/len(t)),
                            'level': target
                            })



0.8389830508474576
0.7372881355932204
0.7288135593220338
0.7457627118644068
0.7457627118644068
0.652542372881356
0.5657894736842105
0.6842105263157895
0.7368421052631579
0.7236842105263158
0.7105263157894737
0.8421052631578947


In [89]:
fig = plt.figure()
y = english.groupby('level1').sentences.count()
plt.bar(y.keys(), y.values)
plt.show()

In [90]:
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

import keras.models
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

data_cleaned = english.loc[:, "sentences":"Yule's K"].T.dropna().T
dist = dist_C2.loc[:, "A1":"C2"]

cols = dist.shape[1] + data_cleaned.shape[1]


model = keras.models.Sequential()
model.add(Dense(512, input_shape=(6,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              
              metrics=['accuracy'])

encoder = LabelEncoder()
encoder.fit(dist_C2.loc[:, "level"])

encode_Y = encoder.transform(dist_C2.loc[:, "level"])
Y = np_utils.to_categorical(encode_Y)

# with tf.device('/gpu:0'):

model.fit(dist, Y, epochs=50, batch_size=32) 

# + données du csv

frames = [dist, data_cleaned]

X = pandas.concat(frames, axis=1)

# model.fit(X, Y, epochs=40, batch_size=32) 

Epoch 1/50


   32/27310 [..............................] - ETA: 18:58 - loss: 2.2479 - acc: 0.1562

  608/27310 [..............................] - ETA: 1:00 - loss: 1.6883 - acc: 0.3224 

 1216/27310 [>.............................] - ETA: 30s - loss: 1.5866 - acc: 0.3281 

 1824/27310 [=>............................] - ETA: 20s - loss: 1.5538 - acc: 0.3306

...

25152/27310 [==========================>...] - ETA: 0s - loss: 0.3270 - acc: 0.8654

25760/27310 [===========================>..] - ETA: 0s - loss: 0.3272 - acc: 0.8651

26368/27310 [===========================>..] - ETA: 0s - loss: 0.3269 - acc: 0.8653

26976/27310 [============================>.] - ETA: 0s - loss: 0.3276 - acc: 0.8648

27310/27310 [==============================] - 2s 88us/step - loss: 0.3280 - acc: 0.8647


In [88]:

data_standard = StandardScaler().fit_transform(X)
corr = np.corrcoef(data_standard.T)
corr.shape


sea.heatmap(corr)
corr


array([[ 1.        ,  0.89394589,  0.8852928 , ..., -0.07623104,
        -0.00450602, -0.04707961],
       [ 0.89394589,  1.        ,  0.94010929, ..., -0.16480469,
        -0.01736186, -0.09272521],
       [ 0.8852928 ,  0.94010929,  1.        , ..., -0.18961026,
        -0.0150832 , -0.12758151],
       ...,
       [-0.07623104, -0.16480469, -0.18961026, ...,  1.        ,
         0.6858437 , -0.48637956],
       [-0.00450602, -0.01736186, -0.0150832 , ...,  0.6858437 ,
         1.        , -0.5895272 ],
       [-0.04707961, -0.09272521, -0.12758151, ..., -0.48637956,
        -0.5895272 ,  1.        ]])

In [13]:
from sklearn.model_selection import cross_val_score
%time listerr = cross_val_score(clf, dist_C2, target, cv=5)


NameError: name 'clf' is not defined